## Window functions in PySpark

Window functions in PySpark are used to perform operations on a subset of data (or a "window") defined by one or more columns, without collapsing the dataset into an aggregated result. 

#### Common Types of Window Functions
- **Ranking Functions:** E.g., row_number(), rank(), dense_rank()
- **Aggregate Functions:** E.g., avg(), sum(), max(), min(), count()
- **Analytical Functions:** E.g., lead(), lag(), ntile(), cume_dist()

**Steps to Use Window Functions**
- **Import Required Modules**
  ```python
  from pyspark.sql import SparkSession
  from pyspark.sql.window import Window
  from pyspark.sql.functions import *

  ```
- **Define the Window Specification** Use `Window.partitionBy()` to define partitions and `Window.orderBy()` to define the order.
- **Apply the Window Function** Use functions like `row_number(),` `rank()`, or aggregate functions with` over()` to apply the window spec.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

# sample data
emp_data = [(1,'Rohish',50000,'IT','m'),
(2,'Smit',60000,'sales','m'),
(3,'Chetan',70000,'marketing','m'),
(4,'Rajesh',80000,'IT','m'),
(5,'Melody',90000,'sales','f'),
(6,'Babli',45000,'marketing','f'),
(7,'Aish',55000,'marketing','f'),
(8,'Ashwini',100000,'IT','f'),
(9,'Harris',65000,'IT','m'),
(10,'Veeru',50000,'marketing','m'),
(11,'Sradhya',50000,'IT','f'),
(12,'Akshay',90000,'sales','m')]

# schema
schema =  "id int, name string, salary int, department string, gender string"

# employee dataframe
emp_df = spark.createDataFrame(emp_data, schema)
emp_df.show()

+---+-------+------+----------+------+
| id|   name|salary|department|gender|
+---+-------+------+----------+------+
|  1| Rohish| 50000|        IT|     m|
|  2|   Smit| 60000|     sales|     m|
|  3| Chetan| 70000| marketing|     m|
|  4| Rajesh| 80000|        IT|     m|
|  5| Melody| 90000|     sales|     f|
|  6|  Babli| 45000| marketing|     f|
|  7|   Aish| 55000| marketing|     f|
|  8|Ashwini|100000|        IT|     f|
|  9| Harris| 65000|        IT|     m|
| 10|  Veeru| 50000| marketing|     m|
| 11|Sradhya| 50000|        IT|     f|
| 12| Akshay| 90000|     sales|     m|
+---+-------+------+----------+------+



### Ranking Functions

**row_number():** Assigns a unique number to each row in a partition based on the specified order.

In [0]:
window = Window.partitionBy("department").orderBy(desc("salary"))
emp_df.withColumn("rn", row_number().over(window)).show()

+---+-------+------+----------+------+---+
| id|   name|salary|department|gender| rn|
+---+-------+------+----------+------+---+
|  8|Ashwini|100000|        IT|     f|  1|
|  4| Rajesh| 80000|        IT|     m|  2|
|  9| Harris| 65000|        IT|     m|  3|
|  1| Rohish| 50000|        IT|     m|  4|
| 11|Sradhya| 50000|        IT|     f|  5|
|  3| Chetan| 70000| marketing|     m|  1|
|  7|   Aish| 55000| marketing|     f|  2|
| 10|  Veeru| 50000| marketing|     m|  3|
|  6|  Babli| 45000| marketing|     f|  4|
|  5| Melody| 90000|     sales|     f|  1|
| 12| Akshay| 90000|     sales|     m|  2|
|  2|   Smit| 60000|     sales|     m|  3|
+---+-------+------+----------+------+---+



**rank():** Assigns ranks to rows with ties, leaving gaps in ranks.

In [0]:
window = Window.partitionBy("department").orderBy(col("salary").desc())
emp_df.withColumn("rank", rank().over(window)).show()

+---+-------+------+----------+------+----+
| id|   name|salary|department|gender|rank|
+---+-------+------+----------+------+----+
|  8|Ashwini|100000|        IT|     f|   1|
|  4| Rajesh| 80000|        IT|     m|   2|
|  9| Harris| 65000|        IT|     m|   3|
|  1| Rohish| 50000|        IT|     m|   4|
| 11|Sradhya| 50000|        IT|     f|   4|
|  3| Chetan| 70000| marketing|     m|   1|
|  7|   Aish| 55000| marketing|     f|   2|
| 10|  Veeru| 50000| marketing|     m|   3|
|  6|  Babli| 45000| marketing|     f|   4|
|  5| Melody| 90000|     sales|     f|   1|
| 12| Akshay| 90000|     sales|     m|   1|
|  2|   Smit| 60000|     sales|     m|   3|
+---+-------+------+----------+------+----+



**dense_rank():** Assigns ranks without gaps in case of ties.

In [0]:
emp_df.withColumn("dense_rank", dense_rank().over(window)).show()

+---+-------+------+----------+------+----------+
| id|   name|salary|department|gender|dense_rank|
+---+-------+------+----------+------+----------+
|  8|Ashwini|100000|        IT|     f|         1|
|  4| Rajesh| 80000|        IT|     m|         2|
|  9| Harris| 65000|        IT|     m|         3|
|  1| Rohish| 50000|        IT|     m|         4|
| 11|Sradhya| 50000|        IT|     f|         4|
|  3| Chetan| 70000| marketing|     m|         1|
|  7|   Aish| 55000| marketing|     f|         2|
| 10|  Veeru| 50000| marketing|     m|         3|
|  6|  Babli| 45000| marketing|     f|         4|
|  5| Melody| 90000|     sales|     f|         1|
| 12| Akshay| 90000|     sales|     m|         1|
|  2|   Smit| 60000|     sales|     m|         2|
+---+-------+------+----------+------+----------+



#### popular coding questions related to ranking functions

**Retrieve the Nth Highest(top 2) Salary in Each Department**

In [0]:
window = Window.partitionBy("department").orderBy(col("salary").desc())

emp_df.withColumn("dense_rank", dense_rank().over(window)) \
      .filter(col("dense_rank") <= 2).show()

+---+-------+------+----------+------+----------+
| id|   name|salary|department|gender|dense_rank|
+---+-------+------+----------+------+----------+
|  8|Ashwini|100000|        IT|     f|         1|
|  4| Rajesh| 80000|        IT|     m|         2|
|  3| Chetan| 70000| marketing|     m|         1|
|  7|   Aish| 55000| marketing|     f|         2|
|  5| Melody| 90000|     sales|     f|         1|
| 12| Akshay| 90000|     sales|     m|         1|
|  2|   Smit| 60000|     sales|     m|         2|
+---+-------+------+----------+------+----------+



**Remove Duplicate Rows Based on a Column**

In [0]:
# Use row_number() and filter for rows with Row_Number = 1.
window = Window.partitionBy("salary").orderBy("name")
emp_df.withColumn("rn", row_number().over(window)) \
      .filter(col("rn") == 1) \
      .drop(col("rn")).show()

+---+-------+------+----------+------+
| id|   name|salary|department|gender|
+---+-------+------+----------+------+
|  6|  Babli| 45000| marketing|     f|
|  1| Rohish| 50000|        IT|     m|
|  7|   Aish| 55000| marketing|     f|
|  2|   Smit| 60000|     sales|     m|
|  9| Harris| 65000|        IT|     m|
|  3| Chetan| 70000| marketing|     m|
|  4| Rajesh| 80000|        IT|     m|
| 12| Akshay| 90000|     sales|     m|
|  8|Ashwini|100000|        IT|     f|
+---+-------+------+----------+------+



**Find Employees with the Same Rank Across All Departments**

In [0]:
window = Window.partitionBy("department").orderBy(desc("salary"))

emp_df.withColumn("dns_rnk", dense_rank().over(window)) \
    .groupBy("department", "dns_rnk") \
    .agg(count(col("dns_rnk")).alias("count")) \
    .filter(col("count") > 1).show()

+----------+-------+-----+
|department|dns_rnk|count|
+----------+-------+-----+
|        IT|      4|    2|
|     sales|      1|    2|
+----------+-------+-----+



**Find the Department with the Highest Average Rank**

In [0]:
# Identify the department where employees have the highest average rank based on salary.
# Solution:
# Use dense_rank() to rank employees within each department.
# Calculate the average rank for each department.

window = Window.partitionBy("department").orderBy(desc("salary"))

rank_df = emp_df.withColumn("dense_rank", dense_rank().over(window))

rank_df.groupBy("department") \
      .agg(avg(col("dense_rank")).alias("avg_rnk")) \
      .sort(col("avg_rnk")).show()

+----------+------------------+
|department|           avg_rnk|
+----------+------------------+
|     sales|1.3333333333333333|
| marketing|               2.5|
|        IT|               2.8|
+----------+------------------+



In [0]:
# Identify the department where employees have the highest average rank based on salary.
# Solution:
# Use dense_rank() to rank employees within each department.
# Calculate the average rank for each department.

window = Window.partitionBy("department").orderBy(desc("salary"))

rank_df = emp_df.withColumn("dense_rank", dense_rank().over(window))

rank_df.groupBy("department") \
      .agg(avg(col("dense_rank")).alias("avg_rnk")) \
      .sort(col("avg_rnk")).show()

+----------+------------------+
|department|           avg_rnk|
+----------+------------------+
|     sales|1.3333333333333333|
| marketing|               2.5|
|        IT|               2.8|
+----------+------------------+



### Aggregate functions
Aggregate window functions in PySpark allow you to perform computations like `SUM`, `AVG`, `COUNT`, `MAX`, `MIN`, etc., over a specific window (partition) of data.

In [0]:
# Sample DataFrame
data = [
    ("Abhi", "HR", 4000),
    ("Jaya", "HR", 5000),
    ("Charlie", "HR", 4500),
    ("Rohish", "IT", 5500),
    ("Rohan", "IT", 6000),
    ("Ramesh", "Finance", 7000),
    ("Ramu", "Finance", 6500),
]

columns = ["employee", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

+--------+----------+------+
|employee|department|salary|
+--------+----------+------+
|    Abhi|        HR|  4000|
|    Jaya|        HR|  5000|
| Charlie|        HR|  4500|
|  Rohish|        IT|  5500|
|   Rohan|        IT|  6000|
|  Ramesh|   Finance|  7000|
|    Ramu|   Finance|  6500|
+--------+----------+------+



**Cumulative Sum (Running Total)**

In [0]:
# default window frame is rowsBetween(Window.unboundedPreceding, Window.currentRow)
window = Window.orderBy("salary")
df.withColumn("runing_sum", sum("salary").over(window)).show()

+--------+----------+------+----------+
|employee|department|salary|runing_sum|
+--------+----------+------+----------+
|    Abhi|        HR|  4000|      4000|
| Charlie|        HR|  4500|      8500|
|    Jaya|        HR|  5000|     13500|
|  Rohish|        IT|  5500|     19000|
|   Rohan|        IT|  6000|     25000|
|    Ramu|   Finance|  6500|     31500|
|  Ramesh|   Finance|  7000|     38500|
+--------+----------+------+----------+



**Explanation of Frame Parameters**
- `Window.unboundedPreceding:` Includes all rows before the current row in the frame.
- `Window.unboundedFollowing:` Includes all rows after the current row in the frame.
- `Window.currentRow:` Refers to the current row in the frame.

In [0]:
# default window frame is rowsBetween(Window.unboundedPreceding, Window.currentRow)
window = Window.partitionBy("department").orderBy("salary")

df.withColumn("runing_sum", sum("salary").over(window)).show()

+--------+----------+------+----------+
|employee|department|salary|runing_sum|
+--------+----------+------+----------+
|    Ramu|   Finance|  6500|      6500|
|  Ramesh|   Finance|  7000|     13500|
|    Abhi|        HR|  4000|      4000|
| Charlie|        HR|  4500|      8500|
|    Jaya|        HR|  5000|     13500|
|  Rohish|        IT|  5500|      5500|
|   Rohan|        IT|  6000|     11500|
+--------+----------+------+----------+



**Average Salary Per Partition**

In [0]:
window = Window.partitionBy("department")

df.withColumn("avg_salary", avg("salary").over(window)).show()

+--------+----------+------+----------+
|employee|department|salary|avg_salary|
+--------+----------+------+----------+
|  Ramesh|   Finance|  7000|    6750.0|
|    Ramu|   Finance|  6500|    6750.0|
|    Abhi|        HR|  4000|    4500.0|
|    Jaya|        HR|  5000|    4500.0|
| Charlie|        HR|  4500|    4500.0|
|  Rohish|        IT|  5500|    5750.0|
|   Rohan|        IT|  6000|    5750.0|
+--------+----------+------+----------+



**Maximum and Minimum Salary**

In [0]:
window = Window.partitionBy("department")

df.withColumn("max_salary", max("salary").over(window)) \
  .withColumn("min_salary", min("salary").over(window)) \
  .show()

+--------+----------+------+----------+----------+
|employee|department|salary|max_salary|min_salary|
+--------+----------+------+----------+----------+
|  Ramesh|   Finance|  7000|      7000|      6500|
|    Ramu|   Finance|  6500|      7000|      6500|
|    Abhi|        HR|  4000|      5000|      4000|
|    Jaya|        HR|  5000|      5000|      4000|
| Charlie|        HR|  4500|      5000|      4000|
|  Rohish|        IT|  5500|      6000|      5500|
|   Rohan|        IT|  6000|      6000|      5500|
+--------+----------+------+----------+----------+



### Analytical Functions

**lead()**
- Fetch the value of a column from the next row in the same partition.

In [0]:
window = Window.partitionBy("department").orderBy("Salary")

df.withColumn("next_salary", lead("salary").over(window)).show()

+--------+----------+------+-----------+
|employee|department|salary|next_salary|
+--------+----------+------+-----------+
|    Ramu|   Finance|  6500|       7000|
|  Ramesh|   Finance|  7000|       null|
|    Abhi|        HR|  4000|       4500|
| Charlie|        HR|  4500|       5000|
|    Jaya|        HR|  5000|       null|
|  Rohish|        IT|  5500|       6000|
|   Rohan|        IT|  6000|       null|
+--------+----------+------+-----------+



**lag()**
- Fetch the value of a column from the previous row in the same partition.

In [0]:
window = Window.partitionBy("department").orderBy("Salary")

df.withColumn("next_salary", lag("salary").over(window)).show()

+--------+----------+------+-----------+
|employee|department|salary|next_salary|
+--------+----------+------+-----------+
|    Ramu|   Finance|  6500|       null|
|  Ramesh|   Finance|  7000|       6500|
|    Abhi|        HR|  4000|       null|
| Charlie|        HR|  4500|       4000|
|    Jaya|        HR|  5000|       4500|
|  Rohish|        IT|  5500|       null|
|   Rohan|        IT|  6000|       5500|
+--------+----------+------+-----------+

